# LLMs Approach

## PIPs and Libraries

In [1]:
COLAB =True # IF YOU USE GOOGLE COLAB -> COLAB = True
PIP = True # IF YOU NEED INSTALL LIBRARIES -> PIP = True

In [ ]:
if PIP:
    !pip install transformers --upgrade
    !pip install datasets accelerate
    !pip install evaluate
    !pip install -U PyEvALL

    !pip install torch
    !pip install numpy
    !pip install pandas
    !pip install scikit-learn
    !pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import sys
import tempfile
import ast

import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import f1_score

from transformers import AutoTokenizer, AutoModelForCausalLM

from pyevall.evaluation import PyEvALLEvaluation
from pyevall.metrics.metricfactory import MetricFactory
from pyevall.reports.reports import PyEvALLReport
from pyevall.utils.utils import PyEvALLUtils


## Drive and Dataset

In [ ]:
from pathlib import Path

if COLAB is True:
  from google.colab import drive
  drive.mount('/content/drive',force_remount=True)
  base_path = "/content/drive/MyDrive/EXISTS2025_TweetBusters"
  library_path = base_path + "/Functions"
else:
  base_path = Path.cwd().parent
  library_path = base_path / "Functions"



sys.path.insert(0, str(library_path))
from readerEXIST2025_2 import EXISTReader

Mounted at /content/drive


In [ ]:
# path to the dataset, adapt this path wherever you have the dataset
dataset_path = os.path.join(base_path, "Dataset/EXIST_2025_Dataset_V0.3/")

file_train = os.path.join(dataset_path, "EXIST2025_training.json")
file_dev = os.path.join(dataset_path, "EXIST2025_dev.json")
file_test = os.path.join(dataset_path, "EXIST2025_test_clean.json")


reader_train = EXISTReader(file_train)
reader_dev = EXISTReader(file_dev)
reader_test = EXISTReader(file_test)


EnTrainTask1, EnDevTask1, EnTestTask1 = reader_train.get(lang="EN", subtask="1"), reader_dev.get(lang="EN", subtask="1"), reader_test.get(lang="EN", subtask="1")
EnTrainTask2, EnDevTask2, EnTestTask2 = reader_train.get(lang="EN", subtask="2"), reader_dev.get(lang="EN", subtask="2"), reader_test.get(lang="EN", subtask="2")
EnTrainTask3, EnDevTask3, EnTestTask3 = reader_train.get(lang="EN", subtask="3"), reader_dev.get(lang="EN", subtask="3"), reader_test.get(lang="EN", subtask="3")


SpTrainTask1, SpDevTask1, SpTestTask1  = reader_train.get(lang="ES", subtask="1"), reader_dev.get(lang="ES", subtask="1"), reader_test.get(lang="ES", subtask="1")
SpTrainTask2, SpDevTask2, SpTestTask2  = reader_train.get(lang="ES", subtask="2"), reader_dev.get(lang="ES", subtask="2"), reader_test.get(lang="ES", subtask="2")
SpTrainTask3, SpDevTask3, SpTestTask3  = reader_train.get(lang="ES", subtask="3"), reader_dev.get(lang="ES", subtask="3"), reader_test.get(lang="ES", subtask="3")


## Import Code Functions

In [ ]:
import os
import importlib.util
import sys
import inspect

functions_path = library_path

for filename in os.listdir(functions_path):
    if filename.endswith(".py") and not filename.startswith("__"):
        module_name = filename[:-3]
        file_path = os.path.join(functions_path, filename)

        # Cargar el módulo
        spec = importlib.util.spec_from_file_location(module_name, file_path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)

        # Extraer todas las funciones del módulo y cargarlas al espacio global
        for name, func in inspect.getmembers(module, inspect.isfunction):
            globals()[name] = func  # o locals()[name] si estás dentro de una función

In [ ]:
model_paths = ["meta-llama/Llama-2-7b-chat-hf","meta-llama/Meta-Llama-3-8B","gpt2"]
unbearalbe_models =["meta-llama/Meta-Llama-3-70B"] #High capacitycomputers



In [ ]:
tokenizer,model = get_model(model_paths[0])
resultados=[]
for languaje in ["En", "Sp"]:
  for task in ["1", "2", "3"]:
    params = dict()
    eval(f"incontext_zero_pipeline_task{task}")(model, tokenizer, eval(f"{languaje}DevTask{task}"), eval(f"{languaje}TestTask{task}"), eval(f"output_postprocessing_incontext_zero_s{task}"))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ID:  400001 Ground Truth: NO Predicted:  YES
ID:  400002 Ground Truth: YES Predicted:  YES
ID:  400003 Ground Truth: YES Predicted:  YES
ID:  400004 Ground Truth: NO Predicted:  YES
ID:  400005 Ground Truth: NO Predicted:  YES
ID:  400006 Ground Truth: NO Predicted:  YES
ID:  400007 Ground Truth: NO Predicted:  YES
ID:  400008 Ground Truth: YES Predicted:  YES
ID:  400009 Ground Truth: NO Predicted:  YES
ID:  400011 Ground Truth: YES Predicted:  YES
ID:  400012 Ground Truth: NO Predicted:  UNK
ID:  400013 Ground Truth: YES Predicted:  YES
ID:  400015 Ground Truth: NO Predicted:  UNK
ID:  400017 Ground Truth: NO Predicted:  YES
ID:  400018 Ground Truth: YES Predicted:  YES
ID:  400020 Ground Truth: YES Predicted:  YES
ID:  400021 Ground Truth: YES Predicted:  YES
ID:  400022 Ground Truth: NO Predicted:  UNK
ID:  400023 Ground Truth: NO Predicted:  UNK
ID:  400024 Ground Truth: NO Predicted:  YES
ID:  400025 Ground Truth: NO Predicted:  NO
ID:  400026 Ground Truth: NO Predicted:  UNK
ID:

In [ ]:
tokenizer,model = get_model(model_paths[0])
resultados=[]
for languaje in ["En", "Sp"]:
  for task in ["1", "2", "3"]:
    params = dict()
    eval(f"incontext_few_pipeline_task{task}")(model, tokenizer, eval(f"{languaje}TrainTask{task}"), eval(f"{languaje}DevTask{task}"), eval(f"{languaje}TestTask{task}"), eval(f"output_postprocessing_incontext_zero_s{task}"))


In [ ]:
tokenizer,model = get_model(model_paths[1])
resultados=[]
for languaje in ["En", "Sp"]:
  for task in ["1", "2", "3"]:
    params = dict()
    eval(f"incontext_zero_pipeline_task{task}")(model, tokenizer, eval(f"{languaje}DevTask{task}"), eval(f"{languaje}TestTask{task}"), eval(f"output_postprocessing_incontext_zero_s{task}"))


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


ID:  400032 Ground Truth: ['IDEOLOGICAL-INEQUALITY'] Predicted:  ['IDEOLOGICAL-INEQUALITY']


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


ID:  400033 Ground Truth: ['IDEOLOGICAL-INEQUALITY', 'STEREOTYPING-DOMINANCE'] Predicted:  ['IDEOLOGICAL-INEQUALITY', 'OBJECTIFICATION']


KeyboardInterrupt: 

In [ ]:
tokenizer,model = get_model(model_paths[2])
resultados=[]
for languaje in ["En", "Sp"]:
  for task in ["1", "2", "3"]:
    params = dict()
    eval(f"incontext_zero_pipeline_task{task}")(model, tokenizer, eval(f"{languaje}DevTask{task}"), eval(f"{languaje}TestTask{task}"), eval(f"output_postprocessing_incontext_zero_s{task}"))
